In [2]:
import pandas as pd
import os

In [3]:
os.getcwd()

'C:\\Users\\olubl\\OneDrive\\14th grade\\PDEW_Predict'

In [6]:
#let's read in the P
all_pdew = pd.read_csv('C:\\Users\\olubl\\OneDrive\\14th grade\\PDEW_Predict\\data\\pdew_3q_2024.csv')
all_pdew = all_pdew[['Year','quarter','city1','city2','nsmiles','passengers','fare','carrier_lg']]

,Year,quarter,city1,city2,nsmiles,passengers,fare,carrier_lg
0,2024,3,"Allentown/Bethlehem/Easton, PA","Dallas/Fort Worth, TX",1307,18,367.69,AA
1,2024,3,"Allentown/Bethlehem/Easton, PA","Denver, CO",1539,89,138.60,G4
2,2024,3,"Allentown/Bethlehem/Easton, PA","Atlanta, GA (Metropolitan Area)",692,107,319.69,DL
3,2024,3,"Allentown/Bethlehem/Easton, PA","Austin, TX",1441,13,256.48,AA
4,2024,3,"Allentown/Bethlehem/Easton, PA","Nashville, TN",685,46,130.88,G4
